# Lanuv: Excel into Pandas 
[http://hayd.github.io/2013/pandas-json/]

In [11]:
import pandas as pd
import numpy as np
import re
import json
import requests

In [12]:
f = 'Anfrage_Rentmeister/Anfrage_Rentmeister.xlsx'
df = pd.read_excel(f, sheetname='NO2_JMW')
df.shape

(153, 11)

In [13]:
df.columns
# df.sort_values(by='2014')

Index([   'Stationscode',    'Stationsname',      'Schadstoff',
               'Einheit',       'dataGroup',              2014,
                    2015, 'TYPE_OF_STATION',    'TYPE_OF_AREA',
                 'Länge',          'Breite'],
      dtype='object')

In [14]:
df.head(1)

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,2014,2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite
0,DENW002,Datteln-Hagem,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,21.1261,20.3173,Hintergrund,vorstädtisches Gebiet,7.331405,51.641383


In [15]:
# Städte rausziehen 

stadt = df['Stationsname'].str.split(' ').str.get(0)
stadt2 = stadt.str.split('-').str.get(0)

df['Stadt'] = stadt2
df['Stadt'].tail()

148    Düsseldorf
149     Paderborn
150       Bottrop
151      Schwerte
152         Herne
Name: Stadt, dtype: object

In [16]:
df.columns

Index([   'Stationscode',    'Stationsname',      'Schadstoff',
               'Einheit',       'dataGroup',              2014,
                    2015, 'TYPE_OF_STATION',    'TYPE_OF_AREA',
                 'Länge',          'Breite',           'Stadt'],
      dtype='object')

In [17]:
df.columns = ['Stationscode', 'Stationsname','Schadstoff', 'Einheit', 'dataGroup', 'Jahr-2014', 'Jahr-2015', 'TYPE_OF_STATION', 'TYPE_OF_AREA', 'Länge', 'Breite', 'Stadt']

In [18]:
df.head(2)

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,Jahr-2014,Jahr-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite,Stadt
0,DENW002,Datteln-Hagem,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,21.1261,20.3173,Hintergrund,vorstädtisches Gebiet,7.331405,51.641383,Datteln
1,DENW006,Lünen-Niederaden,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,27.438,24.3654,Hintergrund,vorstädtisches Gebiet,7.569838,51.592206,Lünen


In [19]:
# df.columns
# df = df.drop(['Stationscode','Stationsname', 'Schadstoff', 'Einheit', 'dataGroup', 'Jahr-2014', 'Jahr-2015', 'TYPE_OF_STATION', 'TYPE_OF_AREA', ] , axis=1)
df.index = df['Stationsname']
df.head()

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,Jahr-2014,Jahr-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite,Stadt
Stationsname,,,,,,,,,,,,
Datteln-Hagem,DENW002,Datteln-Hagem,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,21.1261,20.3173,Hintergrund,vorstädtisches Gebiet,7.331405,51.641383,Datteln
Lünen-Niederaden,DENW006,Lünen-Niederaden,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,27.438,24.3654,Hintergrund,vorstädtisches Gebiet,7.569838,51.592206,Lünen
Dortmund-Eving,DENW008,Dortmund-Eving,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,27.6639,26.5309,Hintergrund,städtisches Gebiet,7.457463,51.536911,Dortmund
Unna-Königsborn,DENW010,Unna-Königsborn,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,20.9525,20.9618,Hintergrund,vorstädtisches Gebiet,7.693835,51.547495,Unna
Bottrop-Welheim,DENW021,Bottrop-Welheim,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,27.3434,26.3245,Industrie,städtisches Gebiet,6.976908,51.525963,Bottrop


In [ ]:
bindestrich_2014 = df[df['Jahr-2014']== '-']
bindestrich_2014.shape
# 2014 13 Stationen mit '-'

In [20]:
bindestrich_2015 = df[df['Jahr-2015']== '-']
bindestrich_2015.shape

# 2015 11 Stationen mit '-'

(11, 12)

In [21]:
df[(df['Jahr-2014'] == '-') & (df['Jahr-2015'] == '-')]
# Keine Station 2014 und 2015 mit '-'

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,Jahr-2014,Jahr-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite,Stadt
Stationsname,,,,,,,,,,,,


In [22]:
clean_year = df[['Jahr-2014', 'Jahr-2015']]

In [23]:
clean_year['Jahr-2014'] = pd.to_numeric(clean_year['Jahr-2014'], errors='coerce')

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
clean_year['Jahr-2015'] = pd.to_numeric(clean_year['Jahr-2015'], errors='coerce')
# clean_year.applymap(lambda x: isinstance(x, (int, float)))clean_year = pd.to_numeric(clean_year['Jahr-2015'], errors='coerce')
# clean_year.applymap(lambda x: isinstance(x, (int, float)))

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
clean_year.dtypes

Jahr-2014    float64
Jahr-2015    float64
dtype: object

In [26]:
clean_year['Jahr-2014'] = clean_year['Jahr-2014'].where(clean_year['Jahr-2014'] > 0, np.nan)
clean_year['Jahr-2014'][0]

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


21.126100000000001

In [27]:
clean_year['Jahr-2015'] = clean_year['Jahr-2015'].where(clean_year['Jahr-2015'] > 0, np.nan)
clean_year['Jahr-2015'][0]

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


20.317299999999999

In [28]:
df['c-2014'] = clean_year['Jahr-2014']
df['c-2015'] = clean_year['Jahr-2015']

In [29]:
# Grenzwert 40 überschritten?

drueber = df[df['c-2014'] >40]
drueber.shape

# 2014 wurde der Grenzwert an 64 Messstationen überschritten 

(64, 14)

In [30]:
# Welches sind die Stationen mit den höchsten Messwerten?
drueber_s = drueber.sort_values(by='c-2014', ascending = False)


In [41]:
# 2015 Höchste Grenzwerte?
top_2015 = df.sort_values(by='c-2015', ascending=False)
drueber_2015 = top_2015[top_2015['c-2015'] >40]
drueber_2015_staedte = drueber_2015.groupby('Stadt').agg(['count'])
drueber_2015_staedte.sort_values(by=[('Stationsname','count')], ascending=False).head(5)

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,Jahr-2014,Jahr-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite,c-2014,c-2015
,count,count,count,count,count,count,count,count,count,count,count,count,count
Stadt,,,,,,,,,,,,,
Köln,10,10,10,10,10,10,10,10,10,10,10,9,10
Essen,6,6,6,6,6,6,6,6,6,6,6,6,6
Düsseldorf,4,4,4,4,4,4,4,4,4,4,4,4,4
Dortmund,4,4,4,4,4,4,4,4,4,4,4,4,4
Paderborn,3,3,3,3,3,3,3,3,3,3,3,2,3


In [45]:
# Wie viele Messstellen wo? 

mess = df.groupby('Stadt').agg(['count'])
mess.sort_values(by=[('Stationsname','count')], ascending=False).head(5)


,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,Jahr-2014,Jahr-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite,c-2014,c-2015
,count,count,count,count,count,count,count,count,count,count,count,count,count
Stadt,,,,,,,,,,,,,
Düsseldorf,17,17,17,17,17,17,17,17,17,17,17,7,7
Köln,15,15,15,15,15,15,15,15,15,15,15,14,14
Essen,13,13,13,13,13,13,13,13,13,13,13,13,12
Dortmund,6,6,6,6,6,6,6,6,6,6,6,6,6
Duisburg,6,6,6,6,6,6,6,6,6,6,6,6,6


In [ ]:
# Next step: Anteil der Stationen mit überschrittenem Grenzwert an der Anzahl der Stationen berechnen

In [ ]:
# df = df.drop(['year_2014','year_2015'], axis=1)

In [50]:
no2 = df[['Stadt', 'Schadstoff', 'Einheit', 'c-2014', 'c-2015', 'TYPE_OF_STATION', 'TYPE_OF_AREA', 'Länge', 'Breite']]
no2.columns

Index(['Stadt', 'Schadstoff', 'Einheit', 'c-2014', 'c-2015', 'TYPE_OF_STATION',
       'TYPE_OF_AREA', 'Länge', 'Breite'],
      dtype='object')

In [52]:
no2.head()

,Stadt,Schadstoff,Einheit,c-2014,c-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite
Stationsname,,,,,,,,,
Datteln-Hagem,Datteln,Stickstoffdioxid,µg/m³,21.1261,20.3173,Hintergrund,vorstädtisches Gebiet,7.331405,51.641383
Lünen-Niederaden,Lünen,Stickstoffdioxid,µg/m³,27.4380,24.3654,Hintergrund,vorstädtisches Gebiet,7.569838,51.592206
Dortmund-Eving,Dortmund,Stickstoffdioxid,µg/m³,27.6639,26.5309,Hintergrund,städtisches Gebiet,7.457463,51.536911
Unna-Königsborn,Unna,Stickstoffdioxid,µg/m³,20.9525,20.9618,Hintergrund,vorstädtisches Gebiet,7.693835,51.547495
Bottrop-Welheim,Bottrop,Stickstoffdioxid,µg/m³,27.3434,26.3245,Industrie,städtisches Gebiet,6.976908,51.525963


In [55]:
no2.columns = ['Stadt', 'Schadstoff', 'Einheit', 'c-2014', 'c-2015', 'Stationstyp','Gebiet', 'x', 'y']
no2.tail()

,Stadt,Schadstoff,Einheit,c-2014,c-2015,Stationstyp,Gebiet,x,y
Stationsname,,,,,,,,,
Düsseldorf Norfer Straße,Düsseldorf,Stickstoffdioxid,µg/m³,NaN,31.8203,Hintergrund,städtisches Gebiet,6.761111,51.200306
Paderborn-Schloss Neuhaus,Paderborn,Stickstoffdioxid,µg/m³,NaN,41.5547,Verkehr,städtisches Gebiet,8.711944,51.743583
Bottrop Peterstraße 9,Bottrop,Stickstoffdioxid,µg/m³,NaN,37.7425,Verkehr,städtisches Gebiet,6.924361,51.519167
Schwerte Hörder Straße 13,Schwerte,Stickstoffdioxid,µg/m³,NaN,45.9835,Verkehr,städtisches Gebiet,7.563578,51.446714
Herne Recklinghauser Straße 4/6,Herne,Stickstoffdioxid,µg/m³,NaN,NaN,Verkehr,städtisches Gebiet,7.150192,51.543008


In [56]:
df.to_csv('no2_orte.csv')